<h1><center>Biogen Pretrained Tutorial - independent version</center></h1>


<center>Author: Qihuang Zhang*, Jian Hu, Kejie Li, Baohong Zhang, David Dai, Edward B. Lee, Rui Xiao, Mingyao Li*

### Outline
1. Preparation
2. Import modules
3. Data Loading
4. Run CeLEry
5. Data Augmentation

In this tutorial, we illustrate the usage of the CeLEry pretrain model trained by Biogene mouse brain data (Li and Zhang, 2022). This model takes the gene expression input of 886 genes and produce a prediction probability vector to eight regions segemented from the spatial transcriptomics data.

This tutorial can be independent of the CeLEry package. It does not require installing the CeLEry package. 

### 1. Preparation

To implemente the model without installing CeLEry package, several helper functions are needed. The ``pickle`` package is used to load the pretrained model. Function ``make_annData_query()`` transform the raw input data into AnnData format and conduct data proprocessing, including  normalizing the gene expression per cell and performing ``log(1+p)`` transcformation. The ``get_zscore()`` helps to normalized the gene expression so that batch effect can be removed.

In [ ]:
import pickle
from scanpy import read_10x_h5

import scanpy as sc
import numpy as np
from scipy.sparse import issparse


def make_annData_query (adata):
    """ 
    adata: an annData file for the scRNA data
    """
    adata.var_names = [i.upper() for i in list(adata.var_names)]
    adata.var["genename"] = adata.var.index.astype("str")
    #
    adata.var_names_make_unique()
    #Normalize and take log for UMI
    sc.pp.normalize_per_cell(adata)
    sc.pp.log1p(adata)
    return adata



def get_zscore (adata, mean = None, sd = None ):
	genotypedata = (adata.X.A if issparse(adata.X) else adata.X)
	if mean is None:
		genemean = np.mean(genotypedata, axis =0)
		genesd = np.std(genotypedata, axis = 0)
	else:
		genemean = mean
		genesd = sd
	try:
		if adata.standardize is not True:
				datatransform = (genotypedata - genemean) / genesd
				adata.X = datatransform
				adata.genemean = genemean
				adata.genesd = genesd
				adata.standardize = True
		else:
			print("Data has already been z-scored")
	except AttributeError:
		datatransform = (genotypedata - genemean) / genesd
		adata.X = datatransform
		adata.genemean = genemean
		adata.genesd = genesd
		adata.standardize = True
		

### 2. Load Data
 
Load scRNA-seq/snRNA-seq data. Example data can be download from [Li and Zhang (2022)](https://doi.org/10.5281/zenodo.6640285).

In [ ]:

# QueryData_raw = read_10x_h5("/data/Biogene/7G-1/filtered_feature_bc_matrix.h5")
QueryData_raw = read_10x_h5("../data/Biogene/cellranger/7G-1/filtered_feature_bc_matrix.h5")
QueryData = cel.make_annData_query (QueryData_raw)

It is import to make sure the query scRNA-seq/snRNA-seq contains all the gene in the trained model.

In [ ]:
## Load gene list
filename = "../output/Biogene/Reference_genes_8_075B.obj"
filehandler = open(filename, 'rb') 
genenames = pickle.load(filehandler)

## Rearrange the data and filter the selected genes in the trained model.
Qdata = QueryData[:,list(genenames)]
cel.get_zscore(Qdata)

#### 3. Apply Pre-trained CeLEry model to the snRNA data

The gene expression of the first cell (a 1X886 matrix) in the snRNA-seq data is given by:

In [ ]:
Qdata[0].X.A

Load the CeLEry prediction model which is located at the ``"../output/Biogene/models"`` named as ``Org_domain_075B``. We use CeLEry function ``Predict_domain()`` to conduct domain prediction for each single cells in the scRNA-seq/snRNA-seq data. The detailed argument are explained as follows:

* data_test: (AnnData object) the input scRNA-seq/snRNA-seq data 
* class_num: (int) the number of class to be predicted. This value should be consistent with the number of domains in the training model.
* path: (string) the location of the pre-trained model
* filename: (string) the file name of the saved pre-trained model
* predtype: (string) if predtype is "probability" (default) then a probability prediction matrix will be produced; if predtype is "deterministic", then the deterministic assignment based on the maximun probability prediction will be returned; if predtype is "both", then both prediction will be outputed. 

In [ ]:
# filename = "../output/Biogene/models/Org_domain_075B.obj"
# filehandler = open(filename, 'rb') 
# CeLERymodel = pickle.load(filehandler)

## Prediction of the first cell

pred_cord = cel.Predict_domain(data_test = Qdata[0], class_num = 8, path = "../output/Biogene/models", filename = "Org_domain_075B", predtype = "probability")


## Prediction of the entire scRNA-seq data

pred_cord = cel.Predict_domain(data_test = Qdata, class_num = 8, path = "../output/Biogene/models", filename = "Org_domain_075B", predtype = "probability")



#### 1.3 Anaconda
If you do not have Python3.5 or Python3.6 installed, consider installing Anaconda (see Installing Anaconda). After installing Anaconda, you can create a new environment, for example, CeLEry (you can change to any name you like).

In [ ]:
#create an environment called CeLEry
conda create -n CeLEry python=3.7.9
#activate your environment 
conda activate CeLEry
git clone https://github.com/QihuangZhang/CeLEry
cd CeLEry/CeLEry_package/
python3 setup.py build
python3 setup.py install
conda deactivate

### 2. Import python modules

In [1]:
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
from skimage import io, color

from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import pickle

import json
os.chdir("CeLEryPython")
import CeLEryPy as cel


In [ ]:
cel.__version__

### 3. Read in data
The current version of CeLEry takes two input data, the reference data and the query data. The reference data is used to trained the model and the query data is the dataset where predictions (or classifications) are made. 
<br>
1. The Reference Data (the spatial transcriptomics data): AnnData format including   
- the gene expression matrix spot by gene ($n_{spot}$ by $k$);
- the spot-specific information (e.g., coordinates, layer, etc.)
<br>
2. The Query Data (the scRNA-seq data): AnnData format including 
- the gene expression matrix cell by gene ($n_{cell}$ by $k$);
- the demographic information for each cell (e.g., cell type, layer, etc.)
<br>

AnnData stores a data matrix `.X` together with annotations of observations `.obs`, variables `.var` and unstructured annotations `.uns`. 

In [ ]:
"""
#Read original 10x_h5 data and save it to h5ad
from scanpy import read_10x_h5
adata = read_10x_h5("../tutorial/data/151673/expression_matrix.h5")
spatial = pd.read_csv("../tutorial/data/151673/positions.txt",sep=",",header=None,na_filter=False,index_col=0) 
adata.obs["x1"] = spatial[1]
adata.obs["x2"] = spatial[2]
adata.obs["x3"] = spatial[3]
adata.obs["x4"] = spatial[4]
adata.obs["x5"] = spatial[5]
adata.obs["x_array"] = adata.obs["x2"]
adata.obs["y_array"] = adata.obs["x3"]
adata.obs["x_pixel"] = adata.obs["x4"]
adata.obs["y_pixel"] = adata.obs["x5"]
#Select captured samples
adata = adata[adata.obs["x1"]==1]
adata.var_names = [i.upper() for i in list(adata.var_names)]
adata.var["genename"] = adata.var.index.astype("str")
adata.write_h5ad("../tutorial/data/151673/sample_data.h5ad")
"""
#Read in gene expression and spatial location
Qdata = sc.read("../tutorial/data/MouseSCToy.h5ad")
Rdata = sc.read("../tutorial/data/MousePosteriorToy.h5ad")


Before inplementing our methods, we often normalize both the reference data and the query data:

In [ ]:
cel.get_zscore(Qdata)
cel.get_zscore(Rdata)


### 4. Run CeLEry

We demonstrate the implemnetation of CeLEry in two tasks. In the first task, CeLEry is implemented to predict the 2D coordinates for the cells. In the second task, we classify the cells into different layers.

#### 4.1 Analysis Task 1: Coordinates Recovery

In the first task, we train a deep neural network using the reference data, and then apply the trained model to predict the location of the cells (or spots) in the query data. 

##### Training 

First, we train the model using spatial transcriptomic data. The trained model will also automately save as an ``.obj`` file in the specified path.

In [ ]:

model_train = cel.Fit_cord (data_train = Rdata, hidden_dims = [30, 25, 15], num_epochs_max = 500, path = "output/example", filename = "PreOrg_Mousesc")


The fitting function ``Fit_cord`` involves the following parameters:

- data_train (an annotated matrix): the input data

- hidden_dims (a list of length three): the width of the neural network in each layer. In total, three layers are considered in the neural network.

- num_epochs_max: maximum number of epochs considered in the training procedure. 

- path: the directory that saving the model object

- filename: the name of the model object to be saved


##### Prediction 

Then, we apply the trained model to the query data to predict the coordinates of the cells.

In [ ]:
pred_cord = cel.Predict_cord (data_test = Qdata, path = "output/example", filename = "PreOrg_Mousesc")

pred_cord

The prediction function ``Predict_cord`` contains three arguments:
- data_test (an annotated matrix): the input query dat

- path: the directory that saving the model object

- filename: the name of the model object to be saved


The method implementation outputs the 2D coordinates in ``pred_cord``. A ``.csv`` file will also saved with the name "predmatrix".

#### 4.2 Analysis Task 2:  Layer Recovery

In the second task, we use CeLEry to classify the cells into different layers. First, we load the spatial transcriptomics data with annotation for layers together with a single cell RNA data collected from an Alzheimer's study.

In [ ]:
Qdata = sc.read("../tutorial/data/AlzheimerToy.h5ad")
Rdata = sc.read("../tutorial/data/DataLayerToy.h5ad")

cel.get_zscore(Qdata)
cel.get_zscore(Rdata)


The sample size of the spots in each layer could be very different, leading to the poor performance of the classification in some layers. We consider weighting the sample from each layer. A typical way to choose weight is to use $1/sample size$. 

In [ ]:
layer_count =  Rdata.obs["Layer"].value_counts().sort_index()
layer_weight = layer_count[7]/layer_count[0:7]
layer_weights = torch.tensor(layer_weight.to_numpy())

We train the model using the function ``Fit_layer``. The model will returned and also save as an ``.obj`` object to be loaded later. 

In [ ]:
model_train = cel.Fit_layer (data_train = Rdata, layer_weights = layer_weights, layerkey = "Layer", 
                             hidden_dims = [30, 25, 15], num_epochs_max = 500, path = "output/example", filename = "PreOrg_layer")

model_train

### 5. Data Augmentation
Due to the limit sample size of the reference data (i.e, spatial transcriptomic data), we can also implementment an augmentation procedure to enlarge the sample size before implementing CeLEry.

(UNDER CONSTRUCTION)